In [1]:
# Import libraries
import numpy as np
import pandas as pd
from datetime import datetime
import copy

import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Set paths
fig_path = './figures/'
data_path = './data/'

------------------------------------
**Get the companies**
------------------------------------

In [2]:
# Read data

mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")
snp = pd.read_csv(data_path+"snp_allstocks_2015_2019.csv", index_col='Date', parse_dates=True, date_parser=mydateparser)
info = pd.read_csv(data_path+'snp_info.csv', index_col=0)

# https://www.slickcharts.com/sp500
# https://datahub.io/core/s-and-p-500-companies-financials
detailed_info = pd.read_csv(data_path+'constituents-financials.csv', index_col=0)
stocks_sorted = detailed_info.sort_values('Market Cap', ascending=False)['Sector']

In [3]:
stocks_by_sector = {sector: [] for sector in info['GICS Sector'].unique()}

for stock in stocks_sorted.index[:160]:
    if stock in ['PCLN', 'TWX', 'AET', 'MON', 'PX', 'ESRX']:
        continue
    stock = 'BRK-B' if stock == 'BRK.B' else stock
    
    sector = info.set_index('Symbol').loc[stock]['GICS Sector']
    stocks_by_sector[sector].append(stock)
# stocks

In [4]:
# Select top three stocks (by market cap) within each GICS Sector
stocks = {}
for sector in stocks_by_sector:
    stocks[sector] = stocks_by_sector[sector][:3]

In [5]:
# use a subset of stocks instead for easy understanding
tmp = {}
tmp['Information Technology'] = stocks['Information Technology']
tmp['Communication Services'] = stocks['Communication Services']
tmp['Consumer Discretionary'] = stocks['Consumer Discretionary']
tmp['Financials'] = stocks['Financials']

stocks = tmp
stocks

{'Information Technology': ['AAPL', 'MSFT', 'V'],
 'Communication Services': ['GOOGL', 'GOOG', 'FB'],
 'Consumer Discretionary': ['AMZN', 'HD', 'MCD'],
 'Financials': ['JPM', 'BAC', 'WFC']}

------------------------------------
**Get the data**
------------------------------------

In [6]:
sector_col = []
stock_col = []
for sector in stocks:
    for stock in stocks[sector]:
        sector_col.append(sector)
        stock_col.append(stock)

df = pd.DataFrame(columns=[sector_col, stock_col], index=snp.index)
df.head()

Information Technology           Communication Services            \
                             AAPL MSFT    V                  GOOGL GOOG   FB   
Date                                                                           
2015-01-02                    NaN  NaN  NaN                    NaN  NaN  NaN   
2015-01-05                    NaN  NaN  NaN                    NaN  NaN  NaN   
2015-01-06                    NaN  NaN  NaN                    NaN  NaN  NaN   
2015-01-07                    NaN  NaN  NaN                    NaN  NaN  NaN   
2015-01-08                    NaN  NaN  NaN                    NaN  NaN  NaN   

           Consumer Discretionary           Financials            
                             AMZN   HD  MCD        JPM  BAC  WFC  
Date                                                              
2015-01-02                    NaN  NaN  NaN        NaN  NaN  NaN  
2015-01-05                    NaN  NaN  NaN        NaN  NaN  NaN  
2015-01-06                    NaN  NaN  NaN        NaN  NaN  NaN  
2015-01-07                    NaN  NaN  NaN        NaN  NaN  NaN  
2015-01-08                    NaN  NaN  NaN        NaN  NaN  NaN

In [7]:
for sector in stocks:
    for stock in stocks[sector]:
        df.loc[:,(sector, stock)] = snp[stock]
df = df.T
df

Date                          2015-01-02  2015-01-05  2015-01-06  2015-01-07  \
Information Technology AAPL   109.330002  106.250000  106.260002  107.750000   
                       MSFT    46.759998   46.330002   45.650002   46.230000   
                       V       66.254997   64.792503   64.375000   65.237503   
Communication Services GOOGL  529.549988  519.460022  506.640015  505.149994   
                       GOOG   521.937744  511.057617  499.212799  498.357513   
                       FB      78.449997   77.190002   76.150002   76.150002   
Consumer Discretionary AMZN   308.519989  302.190002  295.290009  298.420013   
                       HD     103.430000  101.260002  100.949997  104.410004   
                       MCD     93.260002   92.230003   92.400002   94.010002   
Financials             JPM     62.490002   60.549999   58.980000   59.070000   
                       BAC     17.900000   17.379999   16.860001   16.940001   
                       WFC     54.700001   53.200001   52.090000   52.400002   

Date                          2015-01-08  2015-01-09  2015-01-12  2015-01-13  \
Information Technology AAPL   111.889999  112.010002  109.250000  110.220001   
                       MSFT    47.590000   47.189999   46.599998   46.360001   
                       V       66.112503   65.132500   65.004997   65.195000   
Communication Services GOOGL  506.910004  500.720001  497.059998  501.799988   
                       GOOG   499.928864  493.454498  489.854309  493.464447   
                       FB      78.180000   77.739998   76.720001   76.449997   
Consumer Discretionary AMZN   300.459991  296.929993  291.410004  294.739990   
                       HD     106.720001  104.889999  104.419998  103.730003   
                       MCD     94.360001   93.209999   93.010002   92.830002   
Financials             JPM     60.389999   59.340000   58.830002   58.840000   
                       BAC     17.290001   16.980000   16.680000   16.450001   
                       WFC     53.560001   52.680000   52.049999   51.849998   

Date                          2015-01-14  2015-01-15  ...   2018-12-17  \
Information Technology AAPL   109.800003  106.820000  ...   163.940002   
                       MSFT    45.959999   45.480000  ...   102.889999   
                       V       63.889999   63.282501  ...   131.399994   
Communication Services GOOGL  505.929993  504.010010  ...  1025.650024   
                       GOOG   498.128784  499.043732  ...  1016.530029   
                       FB      76.279999   74.050003  ...   140.190002   
Consumer Discretionary AMZN   293.269989  286.950012  ...  1520.910034   
                       HD     102.639999  101.010002  ...   167.970001   
                       MCD     91.540001   91.379997  ...   180.789993   
Financials             JPM     56.810001   54.990002  ...    99.010002   
                       BAC     16.040001   15.200000  ...    24.469999   
                       WFC     51.250000   50.720001  ...    46.610001   

Date                           2018-12-18   2018-12-19   2018-12-20  \
Information Technology AAPL    166.070007   160.889999   156.830002   
                       MSFT    103.970001   103.690002   101.510002   
                       V       132.660004   131.259995   128.759995   
Communication Services GOOGL  1043.410034  1035.459961  1023.580017   
                       GOOG   1028.709961  1023.010010  1009.409973   
                       FB      143.660004   133.240005   133.399994   
Consumer Discretionary AMZN   1551.479980  1495.079956  1460.829956   
                       HD      170.039993   167.559998   164.160004   
                       MCD     179.710007   179.160004   173.679993   
Financials             JPM      98.540001    97.290001    96.449997   
                       BAC      24.469999    24.180000    24.110001   
                       WFC      46.520000    45.669998    46.040001   

Date                           

------------------------------------
**Represent DataFrame as one column Multi-index data**
------------------------------------
Also standardize individual asset names (e.g. asset_1)

In [8]:
# Represent multi-index pandas dataframe as a tensor
# (Multi-index dataframe with only one column of data)
import itertools

T = 1006
N = 12

sector_idx = []
stock_idx = []
for sector in stocks:
    for i, stock in enumerate(stocks[sector]):
        stock_name = 'asset_' + str(i+1)
        sector_idx.append([sector] * T)
        stock_idx.append([stock_name] * T)

sector_idx = list(itertools.chain.from_iterable(sector_idx))
stock_idx = list(itertools.chain.from_iterable(stock_idx))

time_idx = list(snp.index[:T]) * N


df_tensor = pd.DataFrame(index=[sector_idx, stock_idx, time_idx], columns=['Price'])
df_tensor.index.names = ['Sector', 'Asset', 'Date']

In [9]:
for sector in stocks:
    for i, stock in enumerate(stocks[sector]):
        stock_name = 'asset_' + str(i+1)
        df_tensor['Price'].loc[sector, stock_name] = list(snp[stock])

/Users/atayardimci/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2866: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


In [10]:
df_tensor

Price
Sector                 Asset   Date              
Information Technology asset_1 2015-01-02  109.33
                               2015-01-05  106.25
                               2015-01-06  106.26
                               2015-01-07  107.75
                               2015-01-08  111.89
...                                           ...
Financials             asset_3 2018-12-24    43.6
                               2018-12-26   45.59
                               2018-12-27   45.53
                               2018-12-28   45.78
                               2018-12-31   46.08

[12072 rows x 1 columns]

------------------------------------
**Tensorize the data**
------------------------------------

In [11]:
from hottbox.core import Tensor
from hottbox.pdtools import pd_to_tensor

In [12]:
tensor = pd_to_tensor(df_tensor)

In [13]:
tensor

This tensor is of order 3 and consists of 12072 elements.
Sizes and names of its modes are (4, 3, 1006) and ['Sector', 'Asset', 'Date'] respectively.